In [1]:
%matplotlib inline
import seaborn as sns

# Training a network

## Logging

For most purposes, we are going to want to set our logging level to `INFO`, since some commands are going to run for a long time, and we would like periodic updates.

In [2]:
import logging
from imp import reload
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
#logging.debug('This is a debug message')
#logging.basicConfig(level=logging.INFO)

## Setting up an `HDF5` database

The first step is to create a dataset. This process is mostly abstracted away for you using `a3.Dataset` objects. Your responsibility is to specify how your data is stored and structured. We'll start by specifying some structure. 

### Keys

For the example dataset, there is only a single subject. For this dataset, a combination of a study ID and frame ID is sufficient to pick out a unique datapoint.  We will encode this database key as a list of the form `['study','frame']`. While you might construct a key of additional fields (ex. `subject`), the last element will usually be frame, since that is usually the minimal unit of analysis. 

Note that these names are arbitrary, as long as you are consistant. Of course using informative names is best practice, since `autotres` has some sensible defaults that rely on certain keys; for example, the default code for extracting images from video relies on a `'frame'` key.

In [3]:
keys = ['study', 'frame']

### Types

Next, we need to be able to tell autotres about the types of files we have, the types of data they represent, and what levels of the key heirarchy each piece of data should be associated with. This will mostly be accomplished with regular expressions. We create a dict of data types. The keys of the dict represent data type names. Note that we will want one type for each type of data we want, not for each type of file we have. `autotres` is perfectly happy pulling more than one type of information from a single file. 

These are again arbitrary labels, but autotres can provide some sensible default behaviors for certain labels. Each dict contains information about that type:

#### `'conflict'`: How to deal with multiple files of the same type appearing for same combination of identifiers. 

For example, If I have multiple tracers on my team, then I would expect to have multiple 'trace' files for each combination of 'subject' and 'frame'. In this case, I would like to keep all of the traces for the same image, so that I can do something sensible with it, like interpolate. In this case I will set the value for 'conflict' to 'list'. 

Similarly, if I have conducted multiple studies with my dataset, one looking at coronals and one looking at fricatives, I would expect to have multiple copies of the images for coronal fricatives. However, unlike the instance with multiple traces, `fricative_frame-00042.png` and `coronal_frame-00042.png` should be identical files. I can use the 'hash' option to specify that I should ignore duplicates as long as they are the same, but should raise an exception if they don't.

Finally, there are some situations that I simply expect not to happen. For example, if a single subject is associated with more that one 'audio' file, then perhaps it is most likely that somebdy mislabeled something. In this case, I would not set 'conflict' to anything, and if there is a conflict, autotres will raise an exception automatically

#### `'regex'`: How to associate each file with a combination of heirachical levels.

This is a regular expression that will match a filename in the dataset. We use the `(?P<label>...)` syntax to capture parts of the filename that are informative. Specifically, we need to be able to infer all the relevent heirarchical information from the file name. This should be a left-substring of the keys list. 

Note that the regex is matched to the entire pathname, relative to whatever path we give it (see below). Here, we have also used the `(?x)` flag to allow us to break the regex over multiple lines, and to include comments.

In [4]:
import os
import re

types = {
    'trace': {
        'regex': r"""(?x)
            (?P<study>\d+\w+)              # in the example dataset, a 'study' is encoded in the image name as the substring preceding an '_'
            _(?P<frame>\d+)\.(?:jpg|png)   # the frame number
            \.(?P<tracer>\w+)              # the tracer id
            \.traced\.txt$""",
        'conflict': 'list'
        },
    'image': {
        'regex': r"""(?x)
            (?P<study>\d+\w+)
            _(?P<frame>\d+)
            \.(?P<ext>jpg|png)$""",
        'conflict': 'hash'
        },
    'name': {
        'regex': r"""(?x)
            (?P<fname>(?P<study>\d+\w+)
                _(?P<frame>\d+)
                \.(?P<ext>jpg|png)
            )$""",
        }
    }

### Creating the dataset

We will now set up our dataset. The `roi`, `n_points`, and `scale` `kwargs` will be passed down to the default data extraction callbacks (see [`a3/dataset.py`](../a3/dataset.py) documentation). Custom callbacks can be provided by putting a callable in the `ds.callbacks` dict. These should return a numpy array of type `float32`. 

If you don't have CUDA properly installed, importing a3 will throw some errors about nvcc (nvidia cuda compiler) not being found. This is fine so long as you are fine with only using the CPU (instead of the GPU) to train.

In [5]:
import a3
ds = a3.Dataset('example.hdf5',roi=(140.,320.,250.,580.),n_points=32,scale=1/3)

Using gpu device 0: GeForce GTX 660M (CNMeM is disabled)


The directory containing our data is `example_data`. You can scan multiple directories if you need to, possibly with different type definitions, but watch out for file conflicts! Your key heirarchy should be the same accross calls to `scan_directory`. For large datasets may take a while to complete, since it is doing a full walk of the file heirarchy.

In [6]:
d = 'example_data'
ds.scan_directory(d,types,keys)

04:02:03 INFO:scanning /home/phsyron/Documents/autotres/example_data


At this point, you can inspect what data sources you have by looking at the `ds.sources` dict. This `dict` can get very large, so be cautious about printing the whole of it to `stdout`.

In [7]:
ds.sources.keys()
#ds.sources.items()[0]

dict_keys(['20110826JF', '20110518JF', '20110829PB'])

Once you have your data sources figured out, you can extract that data with ds.read_sources(). The arg here is a set-like object with all of the data types you need. This will take a while, since it is opening and processing a lot of files.

In [8]:
ds.read_sources(['trace','image','name'])

04:02:03 INFO:started reading sources


## Training a network

The rest is easy. Construct an `Autotracer` from your new dataset. The required argument points to a `json` file that specifies a network architechture (see [`example.a3.json`](../examples/example.a3.json) for an example). In order to train, you have to specify a training dataset location. You can also do this later, (or even change datasets mid-training) by using the `a.loadHDF5()` method. Whichever way you load your training data, you can specify a `valid` keyword as well to specify a validation set, but leaving it as the default (`None`) sets aside part of your training data as validation data (no guarantees about randomness). Make sure you use the same ROI as above, or at least the same size.

In [9]:
a = a3.Autotracer('examples/example.a3.json', roi=(140.,320.,250.,580.), train='example.hdf5')
# a.loadHDF5('example2.hdf5')

04:02:04 INFO:initializing model
04:02:04 INFO:No regularization
04:02:05 INFO:compiling theano functions


To train on your dataset, simply call the `train()` method. In reality, training will require thousands of epochs (runs through the entire dataset), but for time we will just train a couple times. Minibatch size can be controlled with the `minibatch` kwarg, which defaults to `512`. If your logging level is set to INFO you will see the training loss and validation loss at the end of each epoch.

In [10]:
a.train(10)

04:02:06 INFO:Training
04:02:07 INFO:Epoch: 1, train_loss=0.134391, valid_loss=0.124383
04:02:07 INFO:Epoch: 2, train_loss=0.129632, valid_loss=0.115120
04:02:07 INFO:Epoch: 3, train_loss=0.122775, valid_loss=0.099547
04:02:07 INFO:Epoch: 4, train_loss=0.106627, valid_loss=0.078033
04:02:07 INFO:Epoch: 5, train_loss=0.089502, valid_loss=0.060613
04:02:07 INFO:Epoch: 6, train_loss=0.078829, valid_loss=0.055247
04:02:07 INFO:Epoch: 7, train_loss=0.071783, valid_loss=0.052921
04:02:08 INFO:Epoch: 8, train_loss=0.068495, valid_loss=0.046695
04:02:08 INFO:Epoch: 9, train_loss=0.061481, valid_loss=0.043375
04:02:08 INFO:Epoch: 10, train_loss=0.053597, valid_loss=0.042743


Make sure you save your network! Saving with compression is highly recommended. If you save as a plain `.json` file, your weights will not be saved, by default.

In [11]:
a.save('example.a3.json.bz2')

## Testing a network

Get the traces for your dataset! This will create a file named `original_test.json` that can be used with the APIL web tracer. The remaining positional arguments are the filenames for the images, the tracer ID, and subject ID.

In [12]:
import h5py
with h5py.File('example.hdf5','r') as h:
    # trace all images used in training
    a.trace(h, 'example_test.json', h['name'],'autotrace_test','001')

This output can be easily inspected using the `json` module:

In [13]:
import json
len(json.load(open('example_test.json', 'r'))['trace-data'])

100

If you want to know your loss on some dataset, the best way for now is to train once using that set as the validation set. Note that if you are usign dropout layers, you have to look at `valid_loss`, since `train_loss` will be non-deterministic. Also note that this will train once epoch first, so will be slightly diferent every time. 

In [14]:
b = a3.Autotracer('example.a3.json.bz2', roi=(140.,320.,250.,580.), train='example.hdf5', valid='example.hdf5')
b.train(1)

04:02:14 INFO:initializing model
04:02:14 INFO:No regularization
04:02:14 INFO:compiling theano functions
04:02:16 INFO:Training
04:02:16 INFO:Epoch: 1, train_loss=0.050038, valid_loss=0.042776
